In [1]:
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime

# Günlere göre URL listesi
days = ["pazartesi", "sali", "carsamba", "persembe", "cuma", "cumartesi", "pazar"]
base_url = "https://www.hurriyet.com.tr/tv-rehberi/tum-programlar/"

# Kullanıcı aracısını tanımla (bot olarak algılanmamak için)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

# Tüm haftalık programları saklayacak bir JSON yapısı
weekly_schedule = {}

for day in days:
    url = base_url + day + "/"  # Gün URL'sini oluştur
    response = requests.get(url, headers=headers)

    # Eğer HTTP isteği başarısızsa devam et
    if response.status_code != 200:
        print(f"Hata! {day.capitalize()} günü için siteye erişilemedi. HTTP Status Code: {response.status_code}")
        continue

    # BeautifulSoup ile HTML'yi parse et
    soup = BeautifulSoup(response.text, "lxml")

    # Tüm programları içeren div'leri bul
    programs = soup.find_all("div", class_="flow-module-col")

    # Günlük program listesi
    daily_schedule = []

    for program in programs:
        title_tag = program.find("h2", class_="column-title")
        time_tag = program.find("span", class_="column-time")
        category = program.get("data-type", "Bilinmiyor")  # Eğer tür belirtilmemişse 'Bilinmiyor' yazsın

        if title_tag and time_tag:
            title = title_tag.get_text(strip=True)
            time = time_tag.get_text(strip=True)

            daily_schedule.append({
                "program": title,
                "time": time,
                "category": category
            })

    # Gün verisini JSON yapısına ekle
    weekly_schedule[day.capitalize()] = daily_schedule

# JSON dosyasını kaydet
today_date = datetime.now().strftime("%Y-%m-%d")
json_filename = f"weekly_tv_schedule_{today_date}.json"

with open(json_filename, "w", encoding="utf-8") as json_file:
    json.dump(weekly_schedule, json_file, ensure_ascii=False, indent=4)

print(f"JSON dosyası başarıyla kaydedildi: {json_filename}")


JSON dosyası başarıyla kaydedildi: weekly_tv_schedule_2025-04-09.json


In [13]:
import json
from datetime import datetime

# JSON dosyasını aç ve oku
today_date = datetime.now().strftime("%Y-%m-%d")
json_filename = f"weekly_tv_schedule_{today_date}.json"

with open(json_filename, "r", encoding="utf-8") as json_file:
    weekly_schedule = json.load(json_file)

# Saatleri dakika bazında sıralamak için yardımcı fonksiyon
def convert_time_to_minutes(time_range):
    """'HH:MM - HH:MM' formatındaki saat aralığını dakika cinsinden döndür"""
    start_time, end_time = time_range.split(" - ")
    start_hours, start_minutes = map(int, start_time.split(":"))
    end_hours, end_minutes = map(int, end_time.split(":"))

    return start_hours * 60 + start_minutes, end_hours * 60 + end_minutes

# Günleri işle ve programları düzenle
new_weekly_schedule = {}

for day, programs in weekly_schedule.items():
    channels = {}  # Kanal ID'leriyle programları gruplamak için
    channel_id = 1
    current_channel_programs = []
    previous_start_time = None  # Önceki programın başlangıç saatini tutar

    for i, program in enumerate(programs):
        start_minutes, end_minutes = convert_time_to_minutes(program["time"])

        # Eğer program 22:00 sonrası başladıysa, bir sonraki güne kaydırma!
        if start_minutes >= 22 * 60:  # 22:00 = 1320 dakika
            assigned_day = day  # Program başladığı güne ait olmalı
        elif start_minutes < 6 * 60:  # 06:00 öncesiyse, önceki güne aittir
            previous_day_index = list(weekly_schedule.keys()).index(day) - 1
            if previous_day_index >= 0:
                assigned_day = list(weekly_schedule.keys())[previous_day_index]
            else:
                assigned_day = day  # Önceki gün yoksa kendisine ekle
        else:
            assigned_day = day

        # Yeni bir kanal mı başlatılmalı?
        if previous_start_time is None or start_minutes < previous_start_time:
            if current_channel_programs:
                channels[str(channel_id)] = current_channel_programs  # Önceki kanalı kaydet
                channel_id += 1
                current_channel_programs = []

        # Programı mevcut kanala ekle
        current_channel_programs.append(program)
        previous_start_time = start_minutes  # Şu anki programın başlangıç saatini kaydet

        # Son programdaysa, kanalı kaydet
        if i == len(programs) - 1:
            channels[str(channel_id)] = current_channel_programs

    # Günlük programları güncellenmiş şekilde JSON'a ekle
    new_weekly_schedule[day] = channels

# Yeni JSON dosyasını kaydet
json_filename_updated = f"weekly_tv_schedule_fixed_{today_date}.json"
with open(json_filename_updated, "w", encoding="utf-8") as json_file:
    json.dump(new_weekly_schedule, json_file, ensure_ascii=False, indent=4)

print(f"Yeni düzeltilmiş JSON dosyası oluşturuldu: {json_filename_updated}")


Yeni düzeltilmiş JSON dosyası oluşturuldu: weekly_tv_schedule_fixed_2025-02-02.json


In [14]:
import json
from datetime import datetime

# JSON dosyasını aç ve oku
today_date = datetime.now().strftime("%Y-%m-%d")
json_filename = f"weekly_tv_schedule_fixed_{today_date}.json"

with open(json_filename, "r", encoding="utf-8") as json_file:
    weekly_schedule = json.load(json_file)

# 25. kanalı silip diğerlerini bir yukarı kaydıran fonksiyon
def fix_channels(schedule):
    for day, channels in schedule.items():
        if "25" in channels:
            del channels["25"]  # 25. kanalı tamamen sil

        # Yeni kanal dizisi oluştur
        new_channels = {}
        new_channel_id = 1

        # Eski kanal sırasına göre numaraları kaydır
        for old_channel_id in sorted(channels.keys(), key=int):  # Kanal ID'lerini sırayla dolaş
            new_channels[str(new_channel_id)] = channels[old_channel_id]  # Eski kanalı yeni numaraya ata
            new_channel_id += 1  # Bir sonraki kanal için numarayı artır

        schedule[day] = new_channels  # Günlük kanalları güncelle

    return schedule

# Güncellenmiş kanal numaralarını ayarla
new_weekly_schedule = fix_channels(weekly_schedule)

# Yeni JSON dosyasını kaydet
json_filename_updated = f"weekly_tv_schedule_fixed_final_{today_date}.json"
with open(json_filename_updated, "w", encoding="utf-8") as json_file:
    json.dump(new_weekly_schedule, json_file, ensure_ascii=False, indent=4)

print(f"25. kanal kaldırıldı, diğerleri kaydırıldı. Yeni JSON kaydedildi: {json_filename_updated}")


25. kanal kaldırıldı, diğerleri kaydırıldı. Yeni JSON kaydedildi: weekly_tv_schedule_fixed_final_2025-02-02.json
